In [54]:
import pandas as pd
import boto3
from datetime import datetime
import subprocess

# Cost Predictions on AWS

https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ce.html



## Get the Cost Data from AWS and save it a CSV in S3

In [44]:
def get_cost_data_from_aws(start_date, end_date):
    client = boto3.client('ce')
    results = client.get_cost_and_usage(
        TimePeriod={
            'Start': start_date,
            'End': datetime.now().strftime('%Y-%m-%d')
            },
            Granularity='DAILY',
            Metrics=[
                'UnblendedCost',
            ],
            GroupBy=[
                {
                    'Type': 'DIMENSION',
                    'Key': 'SERVICE'
                },
            ]
        )
    return results

def get_cost_data_to_pandas(start_date, end_date):
    results = get_cost_data_from_aws(start_date, end_date)
    
    start_date_list = []
    cost_list = []
    service_list = []

    for item in results['ResultsByTime']:
        for i in item['Groups']:
            start_date_list.append(item['TimePeriod']['Start'])
        service_list = service_list + [i['Keys'][0] for i in item['Groups']]
        cost_list = cost_list + [i['Metrics']['UnblendedCost']['Amount'] for i in item['Groups']]
    df = pd.DataFrame(list(zip(start_date_list, cost_list, service_list)), 
                      columns =['start_date', 'costs', 'service'])
    return df

In [47]:
start_date = '2022-09-01'  ## This is the month that class started
end_date = datetime.now().strftime('%Y-%m-%d')

my_cost_data = get_cost_data_to_pandas(start_date, end_date)

In [60]:
def save_to_s3(df, end_date):
    '''
    Upload the CSV to S3
    '''
    # save the df as a csv
    csv = f'cost_data_to_{end_date}.csv'
    df.to_csv(csv, index=False)
    
    # set up the session
    client = boto3.client('s3')

    # upload the file
    bucket = 'cost-management-robords'
    print(f'getting file from {csv}')
    key = f'by_service/{csv}'
    with open(csv, "rb") as f:
        client.upload_fileobj(f, bucket, key)

    # Remove the file from the local filesystem
    command_to_run_rm = ["rm", csv]
    output_rm = subprocess.check_output(command_to_run_rm).decode("utf-8").strip()
    print(output_rm)

    print('File uploaded to S3')
    
    return bucket, key, csv

In [62]:
bucket, key, csv = save_to_s3(my_cost_data, end_date)

getting file from cost_data_to_2022-11-07.csv

File uploaded to S3


# Forecast